In [1]:
#Training with MLP classifier on wav2vec2 features

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, TensorDataset
import matplotlib.pyplot as plt


In [3]:
# === Load merged Wav2Vec2 features with labels ===
df = pd.read_csv(r"C:\Users\hp\Desktop\THE_AI_POWERED_COUGH_ANALYZER\wav2vec2_with_labels.csv")

In [4]:
df.head()

,filename,feat_0,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,...,feat_760,feat_761,feat_762,feat_763,feat_764,feat_765,feat_766,feat_767,base_filename,tb_status
0,1637846163058-recording-1_modclean_A_seg0.wav,-0.026476,0.066998,-0.026330,-0.012994,-0.140130,-0.121104,0.082295,-0.000749,-0.034801,...,0.042189,0.375900,-0.115341,-0.003523,-0.013059,-0.097548,0.011164,0.045216,1637846163058-recording-1,1
1,1637846163058-recording-1_modclean_A_seg0.wav,-0.026476,0.066998,-0.026330,-0.012994,-0.140130,-0.121104,0.082295,-0.000749,-0.034801,...,0.042189,0.375900,-0.115341,-0.003523,-0.013059,-0.097548,0.011164,0.045216,1637846163058-recording-1,1
2,1637846163058-recording-1_modclean_B_seg0.wav,-0.115683,0.074489,0.014545,-0.005770,-0.184433,-0.109778,0.070073,-0.004235,-0.051047,...,0.064345,0.192314,-0.074626,-0.005368,-0.018542,-0.167560,0.044945,0.045402,1637846163058-recording-1,1
3,1637846163058-recording-1_modclean_B_seg0.wav,-0.115683,0.074489,0.014545,-0.005770,-0.184433,-0.109778,0.070073,-0.004235,-0.051047,...,0.064345,0.192314,-0.074626,-0.005368,-0.018542,-0.167560,0.044945,0.045402,1637846163058-recording-1,1
4,1637846854303-recording-1_modclean_B_seg0.wav,-0.036963,0.027261,-0.097449,-0.025534,-0.055059,-0.135565,0.002773,-0.035429,-0.032010,...,-0.086376,0.191788,-0.106577,-0.000759,-0.021116,-0.055957,0.054928,0.021372,1637846854303-recording-1,1


In [6]:
df['tb_status'].value_counts()

tb_status
1    8340
0     493
Name: count, dtype: int64

In [7]:
# === Step 1: Extract Features and Labels ===
features = df[[col for col in df.columns if col.startswith("feat_")]].values
labels = df["tb_status"].values

In [8]:
# === Step 2: Scale the features ===
scaler = StandardScaler()
features = scaler.fit_transform(features)

In [9]:
# === Step 3: Train-test split (stratify to preserve class ratio) ===
X_train, X_val, y_train, y_val = train_test_split(
    features, labels, test_size=0.2, stratify=labels, random_state=42
)


In [10]:
# === Step 4: Convert to PyTorch tensors ===
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.long)

train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)

In [11]:
# === Step 5: Define Deep MLP Classifier ===
class DeepMLP(nn.Module):
    def __init__(self, input_dim):
        super(DeepMLP, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 2)
        )

    def forward(self, x):
        return self.model(x)

In [ ]:
# === Step 6: Class weights for imbalance ===
class_counts = np.bincount(y_train.numpy())
class_weights = 1.0 / torch.tensor(class_counts, dtype=torch.float32)
class_weights = class_weights / class_weights.sum()
criterion = nn.CrossEntropyLoss(weight=class_weights)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DeepMLP(input_dim=features.shape[1]).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)